<a href="https://colab.research.google.com/github/SaketMunda/mnist-digit-recognition/blob/master/recognise_handwritten_digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten Digit Recognition

Deep Learning model for Handwritten Digit Recognition using TensorFlow and Neural Network Techniques

## 1. Problem Definition ⛑

Recognise handwritten digit, from a dataset which contains B&W images of each digit written on 28x28 pixel box, given by [The MNIST DATABASE](http://yann.lecun.com/exdb/mnist/) 